In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("<filapath>")

In [3]:
df.shape

(1615940, 2)

In [4]:
df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Role      1615940 non-null  object
 1   Features  1615940 non-null  object
dtypes: object(2)
memory usage: 24.7+ MB


In [6]:
df.columns

Index(['Role', 'Features'], dtype='object')

In [7]:
df['Role'].value_counts()

Role
Interaction Designer            20580
Network Administrator           17470
User Interface Designer         14036
Social Media Manager            13945
User Experience Designer        13935
                                ...  
Inventory Control Specialist     3342
Budget Analyst                   3335
Clinical Nurse Manager           3324
Social Science Researcher        3321
Paid Advertising Specialist      3306
Name: count, Length: 376, dtype: int64

In [8]:
min_count = 6500 #setting minimum to 6.5k, dropping casses less than 6k
role_counts = df['Role'].value_counts()
dropped_classes = role_counts[role_counts < min_count].index
filtered_df = df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)

In [9]:
df['Role'].value_counts()

Role
Interaction Designer            20580
Network Administrator           17470
User Interface Designer         14036
Social Media Manager            13945
User Experience Designer        13935
                                ...  
Inventory Control Specialist     3342
Budget Analyst                   3335
Clinical Nurse Manager           3324
Social Science Researcher        3321
Paid Advertising Specialist      3306
Name: count, Length: 376, dtype: int64

In [10]:
len(df['Role'].value_counts())

376

In [11]:
df = df.sample(n=10000) #n=10000

In [12]:
df.head(10)

,Role,Features
65883,Mechanical Design Engineer,5 to 12 Years Mechanical Engineer BBA Mechanic...
297104,Social Science Researcher,2 to 8 Years Research Analyst BA Social scienc...
940130,Wedding Coordinator,1 to 8 Years Event Coordinator BA Wedding plan...
59858,Data Entry Specialist,0 to 8 Years Administrative Assistant BCA Data...
1326956,Project Coordinator,0 to 14 Years Executive Assistant BCA Project ...
778045,Design Engineer,0 to 13 Years Structural Engineer MCA Engineer...
1145858,Product Designer,3 to 12 Years Mechanical Designer BBA Product ...
508256,Accounting Controller,4 to 15 Years Financial Controller MBA Account...
92284,Supply Chain Coordinator,2 to 12 Years Purchasing Agent BCA Supply chai...
1027260,Key Account Executive,3 to 8 Years Account Executive B.Com Key accou...


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [14]:
#t-t split
X = df['Features']
y = df['Role']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

In [15]:
#tf id vectorisation
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [16]:
#rf classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_tfidf, y_train)

y_pred = rf_classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [17]:
def cleanResume(txt):
    cleanText = re.sub(r'http\S+\s', ' ', txt)  # links
    cleanText = re.sub(r'RT|cc', ' ', cleanText)
    cleanText = re.sub(r'#\S+\s', ' ', cleanText)  # special characters
    cleanText = re.sub(r'@\S+', ' ', cleanText)
    cleanText = re.sub(r'[%s]' % re.escape(r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub(r'\s+', ' ', cleanText)
    return cleanText

In [18]:
def job_recommendation(resume_text):
    resume_text= cleanResume(resume_text)
    resume_tfidf = tfidf_vectorizer.transform([resume_text])
    predicted_category = rf_classifier.predict(resume_tfidf)[0]
    return predicted_category

In [19]:
import pickle
pickle.dump(rf_classifier,open('models/rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('models/tfidf_vectorizer_job_recommendation.pkl','wb'))